# Overview map and stats

This is a bit messy, but there is a bunch of code producing maps and stats for the technical report.

In [ ]:
import pandas as pd
import geopandas as gpd
import subprocess
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
import numpy as np
import sys, os
sys.path.append(os.getcwd() + '/..')
from utils import mkdir, submission_summary, needs_size_filter, size_filter, plot_map, plot_date_hist, maps, open_tar_shapefile, open_zip_shapefile
import os
import matplotlib.patches as mpatches
from oggm import utils
import csv
import seaborn as sns
import cartopy
import cartopy.crs as ccrs
from matplotlib.image import imread

In [ ]:
# go down from rgi7_scripts/workflow
data_dir = '../../../rgi7_data/'

l4_dir = os.path.join(data_dir, 'l4_rgi7b0_tar')

In [ ]:
rgi = []
for reg in range(1, 20):
    shp = gpd.read_file(utils.get_rgi_region_file(f'{reg:02d}', version='60'))
    shp['area'] = shp.to_crs({'proj':'cea'}).area
    shp['O1Region'] = f'{reg:02d}'
    rgi.append(shp)
rgi6 = pd.concat(rgi)
rgi6 = rgi6.loc[rgi6['Connect'] != 2]
rgi6['area'] = rgi6['area'] * 1e-6

In [ ]:
reg_file = open_zip_shapefile(os.path.join(data_dir, '00_rgi70_regions/00_rgi70_O1Regions.zip'))
sreg_file = open_zip_shapefile(os.path.join(data_dir, '00_rgi70_regions/00_rgi70_O2Regions.zip'))

In [ ]:
for n, s in zip(reg_file.o1region, reg_file.full_name):
    print(f'{n} {s}')

In [ ]:
rgi = []
for reg in range(1, 20):
    shp = open_tar_shapefile(os.path.join(l4_dir, 'RGI2000-v7.0-G-' + reg_file.loc[reg_file.o1region == f'{reg:02d}'].iloc[0].long_code + '.tar.gz'))
    rgi.append(shp)
rgi7 = pd.concat(rgi)

## Not valid

In [ ]:
not_valid6 = ~ rgi6.is_valid
not_valid7 = ~ rgi7.is_valid

In [ ]:
not_valid6.sum(), not_valid7.sum()

In [ ]:
not_valid6.sum() / len(rgi6)

In [ ]:
rgi6['Area'].sum(), rgi6['area'].sum(), rgi7['area_km2'].sum()

## RGI6 RGI7 table for report 

In [ ]:
df = rgi6.groupby('O1Region')[['area']].sum()
df.columns = ['Area RGI6']

In [ ]:
df['Area RGI7'] = rgi7.groupby('o1region')[['area_km2']].sum()

In [ ]:
df['Diff A (%)'] = (df['Area RGI7'] / df['Area RGI6'] - 1) * 100

In [ ]:
df['N RGI6'] = rgi6.groupby('O1Region').count()['area']
df['N RGI7'] = rgi7.groupby('o1region').count()['area_km2']

df['Diff N (%)'] = (df['N RGI7'] / df['N RGI6'] - 1) * 100

In [ ]:
df.loc['20'] = [0] * 6

In [ ]:
ss = df.sum()
ss.name = 'Global'
df = pd.concat([df, ss.to_frame().T])

df['Diff A (%)'] = (df['Area RGI7'] / df['Area RGI6'] - 1) * 100
df['Diff N (%)'] = (df['N RGI7'] / df['N RGI6'] - 1) * 100

In [ ]:
df['Area RGI6'] = df['Area RGI6'].astype(float).round(0).astype(int) 
df['Area RGI7'] = df['Area RGI7'].astype(float).round(0).astype(int) 
df['N RGI6'] = df['N RGI6'].astype(int) 
df['N RGI7'] = df['N RGI7'].astype(int) 

df['Diff A (%)'] = df['Diff A (%)'].round(1).replace(-0, 0).replace(np.NaN, 0)
df['Diff N (%)'] = df['Diff N (%)'].round(1).replace(-0, 0).replace(np.NaN, 0)

In [ ]:
print(f"New in N: {len(rgi7)-len(rgi6)}")

In [ ]:
df

In [ ]:
s1 = rgi7.loc[rgi7.is_rgi6 == 1]
print(f"Same as RGI6: N={len(s1)}, A={int(s1['area_km2'].sum())} km²")

s2 = rgi7.loc[rgi7.is_rgi6 == 0]
print(f"New  in RGI7: N={len(s2)}, A={int(s2['area_km2'].sum())} km²")

print(f"New in % area: {s2['area_km2'].sum() / rgi7['area_km2'].sum()}")

print(f"New in % number: {len(s2) / len(rgi7)}")

In [ ]:
df.index.name = 'Region'

In [ ]:
df_formd = df.copy()
df_formd.index = [f'[](rgi{i}.md)' for i in df_formd.index[:-1]] + ['Global']
df_formd.columns = ['Area<br>RGI 6.0 (km²)', 'Area<br>RGI 7.0 (km²)', 'Area<br>Diff. (%)', 'Count<br>RGI 6.0', 'Count<br>RGI 7.0', 'Count<br>Diff. (%)']
print(df_formd.to_markdown(floatfmt=(".0f",".0f",".0f",".1f",".0f",".0f",".1f")).replace('Global    ', '**Global**'))

In [ ]:
df['Full name'] = list(reg_file.full_name.unique()) + ['']
df['Code'] = ['`' + r + '`' for r in reg_file.long_code.unique()] + ['']

In [ ]:
df

In [ ]:
for_csv = df[['Full name', 'Area RGI6', 'Area RGI7', 'Diff A (%)', 'N RGI6', 'N RGI7', 'Diff N (%)']].copy()
for_csv.columns = ['Full name', 'Area RGI 6.0', 'Area RGI 7.0', 'Diff. Area (%)', 'Count RGI 6.0', 'Count RGI 7.0', 'Diff. Count (%)']
for_csv.reset_index().to_csv('RGI2000-v7.0-G-comparison-rgi6.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)

## Tables Appendix (no RGI6) 

### O1 

In [ ]:
for_csv = df[['Full name', 'Code', 'N RGI7', 'Area RGI7']].copy()
for_csv['Code'] = [s.replace("`", "") for s in for_csv['Code']]

In [ ]:
for_csv.columns = [['full_name', 'long_code', 'count', 'area_km2']]
for_csv.index.name = 'o1region'
for_csv.reset_index().to_csv('RGI2000-v7.0-G-o1region-summary.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
for_print = df[['Full name', 'Code', 'N RGI7', 'Area RGI7']]
for_print.columns = ['Full name', 'Code', 'Count', 'Area (km²)']
print(for_print.to_markdown().replace('Global    ', '**Global**'))

### O2 

In [ ]:
sreg_file = sreg_file.drop_duplicates('o2region')
sreg_file

In [ ]:
dfsr = sreg_file[['o2region', 'full_name', 'long_code']].copy().set_index('o2region')

dfsr.columns = ['Full name', 'Code']

dfsr['Count'] = rgi7.groupby('o2region')['cenlon'].count()
dfsr['Area (km²)'] = rgi7.groupby('o2region')['area_km2'].sum()

ss = dfsr.sum()
ss.name = 'Global'
dfsr = pd.concat([dfsr, ss.to_frame().T])

dfsr.loc['Global', 'Full name'] = ''
dfsr.loc['Global', 'Code'] = ''

dfsr['Area (km²)']  = dfsr['Area (km²)'].astype(float).round(0).fillna(0).astype(int) 
dfsr['Count']  = dfsr['Count'].fillna(0).astype(int) 

dfsr['Code'] = ['`' + r + '`' for r in dfsr.Code]

In [ ]:
len(dfsr)

In [ ]:
print(dfsr.to_markdown())

In [ ]:
dfsr['Code'] = [s.replace("`", "") for s in dfsr['Code']]
dfsr.columns = [['full_name', 'long_code', 'count', 'area_km2']]
dfsr.index.name = 'o2region'
dfsr.reset_index().to_csv('RGI2000-v7.0-G-o2region-summary.csv', index=False, quoting=csv.QUOTE_NONNUMERIC)

## Map 

In [ ]:
import salem
df = salem.read_shapefile(os.path.join(data_dir, '00_rgi70_regions', '00_rgi70_O1Regions', '00_rgi70_O1Regions.shp'))

In [ ]:
%config InlineBackend.figure_format = 'retina'
%alias convert convert

In [ ]:
# with sns.plotting_context('poster'), sns.axes_style('whitegrid'):

fac = 1.3
f = plt.figure(figsize=(12*fac, 7*fac))
ax = plt.axes(projection=ccrs.Robinson())
# ax.stock_img()
fname = '/home/users/fmaussion/www_fmaussion/misc/rgi7_data/l0_support_data/images/natural-earth-1_large2048px.png'
ax.imshow(imread(fname), origin='upper', transform=ccrs.PlateCarree(), 
          extent=[-180, 180, -90, 90])

handles = []

df.to_crs(ccrs.Robinson()).plot(ax=ax, edgecolor='k', facecolor='w', alpha=0.3);

s = rgi7.loc[rgi7.is_rgi6 == 1]
ax.scatter(s.cenlon.values, s.cenlat.values, color='C0', s=1, transform=ccrs.PlateCarree());
handles.append(mpatches.Patch(facecolor='C0', label=f"Same as RGI 6.0"))  #  - N={len(s)}, A={int(s['area'].sum()*1e-6)} km²

s = rgi7.loc[rgi7.is_rgi6 == 0]
ax.scatter(s.cenlon.values, s.cenlat.values, color='C3', s=1, transform=ccrs.PlateCarree());
handles.append(mpatches.Patch(facecolor='C3', label=f"New in RGI 7.0"))

df.plot(ax=ax, transform=ccrs.PlateCarree(), edgecolor='k', facecolor='none', linewidth=1);

did10 = False
for i, r in df.iterrows():
    tx = r.min_x + 4
    ty = r.min_y 
    fs = 16
    t = r.o1region

    if t == '01':
        tx = r.min_x + 35
    if t == '03':
        tx = r.min_x + 15
        ty = r.min_y + 3
    if t == '05':
        tx = r.min_x + 40
    if t == '06':
        ty = r.min_y - 5.5
    if t == '08':
        tx = r.min_x + 20
    if t == '09':
        tx = r.min_x 
    if t == '11':
        tx = r.min_x + 24
    if t == '10':
        tx = r.min_x + 1
        ty = r.min_y + 15
        if not did10:
            did10 = True
            continue
    if t == '13':
        ty = r.min_y + 12
        tx = r.min_x + 30
    if t in ['14', '12', '16', '17']:
        tx = r.min_x 
    if t == '15':
        tx = r.min_x + 30
    if t == '17':
        ty = r.min_y + 26
    if t == '18':
        ty = r.min_y + 10
    if t == '19':
        ty = r.min_y + 10
    if t == '20':
        ty = r.min_y + 1
        tx = r.min_x - 20

    ax.text(tx, ty, t, transform=ccrs.PlateCarree(), 
            ha='left', va='bottom', fontsize=fs)


ax.legend(handles=handles, loc='center left', fontsize=fs);

plt.tight_layout();

plt.savefig('../../../rgi7_data/l3_rgi7a_plots/global_map_wrgi6.png', dpi=300, bbox_inches='tight')

In [ ]:
convert ../../../rgi7_data/l3_rgi7a_plots/global_map_wrgi6.png -resize 20% ../../../rgi7_data/l3_rgi7a_plots/global_map_wrgi6_small.jpeg

In [ ]:
# with sns.plotting_context('poster'), sns.axes_style('whitegrid'):

fac = 1.3
f = plt.figure(figsize=(12*fac, 7*fac))
ax = plt.axes(projection=ccrs.Robinson())
# ax.stock_img()
fname = '/home/users/fmaussion/www_fmaussion/misc/rgi7_data/l0_support_data/images/natural-earth-1_large2048px.png'
ax.imshow(imread(fname), origin='upper', transform=ccrs.PlateCarree(), 
          extent=[-180, 180, -90, 90])
handles = []

df.to_crs(ccrs.Robinson()).plot(ax=ax, edgecolor='k', facecolor='w', alpha=0.3);

s = rgi7
ax.scatter(s.cenlon.values, s.cenlat.values, color='C3', s=1, transform=ccrs.PlateCarree());
handles.append(mpatches.Patch(facecolor='C3', label=f"RGI 7.0 glacier outlines"))

df.plot(ax=ax, transform=ccrs.PlateCarree(), edgecolor='k', facecolor='none', linewidth=1);

did10 = False
for i, r in df.iterrows():
    tx = r.min_x + 4
    ty = r.min_y 
    fs = 16
    t = r.o1region

    if t == '01':
        tx = r.min_x + 35
    if t == '03':
        tx = r.min_x + 15
        ty = r.min_y + 3
    if t == '05':
        tx = r.min_x + 40
    if t == '06':
        ty = r.min_y - 5.5
    if t == '08':
        tx = r.min_x + 20
    if t == '09':
        tx = r.min_x 
    if t == '11':
        tx = r.min_x + 24
    if t == '10':
        tx = r.min_x + 1
        ty = r.min_y + 15
        if not did10:
            did10 = True
            continue
    if t == '13':
        ty = r.min_y + 12
        tx = r.min_x + 30
    if t in ['14', '12', '16', '17']:
        tx = r.min_x 
    if t == '15':
        tx = r.min_x + 30
    if t == '17':
        ty = r.min_y + 26
    if t == '18':
        ty = r.min_y + 10
    if t == '19':
        ty = r.min_y + 10
    if t == '20':
        ty = r.min_y + 1
        tx = r.min_x - 20

    ax.text(tx, ty, t, transform=ccrs.PlateCarree(), 
            ha='left', va='bottom', fontsize=fs)


ax.legend(handles=handles, loc='center left', fontsize=fs);

plt.tight_layout();

plt.savefig('../../../rgi7_data/l3_rgi7a_plots/global_map.png', dpi=300, bbox_inches='tight')

# ax.scatter(dfr.lon.values, dfr.lat.values, edgecolor='k', facecolor='tab:cyan', s=24)

# df.plot(ax=ax, transform=ccrs.PlateCarree(), edgecolor='k', facecolor='w', alpha=0.3);
# df.plot(ax=ax, transform=ccrs.PlateCarree(), edgecolor='k', facecolor='none', linewidth=2);

In [ ]:
convert ../../../rgi7_data/l3_rgi7a_plots/global_map.png -resize 20% ../../../rgi7_data/l3_rgi7a_plots/global_map_small.jpeg

In [ ]:
# with sns.plotting_context('poster'), sns.axes_style('whitegrid'):

fac = 1.3
f = plt.figure(figsize=(12*fac, 7*fac))
ax = plt.axes(projection=ccrs.Robinson())

# ax.stock_img()
fname = '/home/users/fmaussion/www_fmaussion/misc/rgi7_data/l0_support_data/images/natural-earth-1_large2048px.png'
ax.imshow(imread(fname), origin='upper', transform=ccrs.PlateCarree(), 
          extent=[-180, 180, -90, 90])

handles = []

df.to_crs(ccrs.Robinson()).plot(ax=ax, edgecolor='k', facecolor='w', alpha=0.3);

s = rgi6
ax.scatter(s.CenLon.values, s.CenLat.values, color='C3', s=1, transform=ccrs.PlateCarree());
handles.append(mpatches.Patch(facecolor='C3', label=f"RGI 6.0 glacier outlines"))

df.plot(ax=ax, transform=ccrs.PlateCarree(), edgecolor='k', facecolor='none', linewidth=1);

did10 = False
for i, r in df.iterrows():
    tx = r.min_x + 4
    ty = r.min_y 
    fs = 16
    t = r.o1region

    if t == '01':
        tx = r.min_x + 35
    if t == '03':
        tx = r.min_x + 15
        ty = r.min_y + 3
    if t == '05':
        tx = r.min_x + 40
    if t == '06':
        ty = r.min_y - 5.5
    if t == '08':
        tx = r.min_x + 20
    if t == '09':
        tx = r.min_x 
    if t == '11':
        tx = r.min_x + 24
    if t == '10':
        tx = r.min_x + 1
        ty = r.min_y + 15
        if not did10:
            did10 = True
            continue
    if t == '13':
        ty = r.min_y + 12
        tx = r.min_x + 30
    if t in ['14', '12', '16', '17']:
        tx = r.min_x 
    if t == '15':
        tx = r.min_x + 30
    if t == '17':
        ty = r.min_y + 26
    if t == '18':
        ty = r.min_y + 10
    if t == '19':
        ty = r.min_y + 10
    if t == '20':
        ty = r.min_y + 1
        tx = r.min_x - 20

    ax.text(tx, ty, t, transform=ccrs.PlateCarree(), 
            ha='left', va='bottom', fontsize=fs)


ax.legend(handles=handles, loc='center left', fontsize=fs);

plt.tight_layout();

plt.savefig('../../../rgi7_data/l3_rgi7a_plots/global_map_rgi6.png', dpi=300, bbox_inches='tight')

In [ ]:
convert ../../../rgi7_data/l3_rgi7a_plots/global_map_rgi6.png -resize 20% ../../../rgi7_data/l3_rgi7a_plots/global_map_rgi6_small.jpeg

In [ ]:
# with sns.plotting_context('poster'), sns.axes_style('whitegrid'):

fac = 1.3
f = plt.figure(figsize=(12*fac, 7*fac))
ax = plt.axes(projection=ccrs.Robinson())
# ax.stock_img()
fname = '/home/users/fmaussion/www_fmaussion/misc/rgi7_data/l0_support_data/images/natural-earth-1_large2048px.png'
ax.imshow(imread(fname), origin='upper', transform=ccrs.PlateCarree(), 
          extent=[-180, 180, -90, 90])
handles = []

df.to_crs(ccrs.Robinson()).plot(ax=ax, edgecolor='k', facecolor='w', alpha=0.3);

for cat, col, name in zip([0, 1, 2, 3], ['royalblue', 'C2', 'C1', 'C3'], ['No evidence', 'Possible', 'Probable', 'Observed']):
    s = rgi7.loc[rgi7.surge_type == cat]
    if name == 'No evidence':
        ax.scatter(s.cenlon.values, s.cenlat.values, fc=col, ec='None', s=1, transform=ccrs.PlateCarree(), linewidth=0);
    else:
        ax.scatter(s.cenlon.values, s.cenlat.values, fc=col, ec='k', s=15, transform=ccrs.PlateCarree(), linewidth=0.5);
        
    handles.append(mpatches.Patch(facecolor=col, label=name))


df.plot(ax=ax, transform=ccrs.PlateCarree(), edgecolor='k', facecolor='none', linewidth=1);

did10 = False
for i, r in df.iterrows():
    tx = r.min_x + 4
    ty = r.min_y 
    fs = 16
    t = r.o1region

    if t == '01':
        tx = r.min_x + 35
    if t == '03':
        tx = r.min_x + 15
        ty = r.min_y + 3
    if t == '05':
        tx = r.min_x + 40
    if t == '06':
        ty = r.min_y - 5.5
    if t == '08':
        tx = r.min_x + 20
    if t == '09':
        tx = r.min_x 
    if t == '11':
        tx = r.min_x + 24
    if t == '10':
        tx = r.min_x + 1
        ty = r.min_y + 15
        if not did10:
            did10 = True
            continue
    if t == '13':
        ty = r.min_y + 12
        tx = r.min_x + 30
    if t in ['14', '12', '16', '17']:
        tx = r.min_x 
    if t == '15':
        tx = r.min_x + 30
    if t == '17':
        ty = r.min_y + 26
    if t == '18':
        ty = r.min_y + 10
    if t == '19':
        ty = r.min_y + 10
    if t == '20':
        ty = r.min_y + 1
        tx = r.min_x - 20

    ax.text(tx, ty, t, transform=ccrs.PlateCarree(), 
            ha='left', va='bottom', fontsize=fs)


legend = ax.legend(handles=handles, loc='center left', fontsize=fs, title='Surge type');
plt.setp(legend.get_title(),fontsize=fs+2)

plt.tight_layout();

plt.savefig('../../../rgi7_data/l3_rgi7a_plots/global_map_surge.png', dpi=300, bbox_inches='tight')

# ax.scatter(dfr.lon.values, dfr.lat.values, edgecolor='k', facecolor='tab:cyan', s=24)

# df.plot(ax=ax, transform=ccrs.PlateCarree(), edgecolor='k', facecolor='w', alpha=0.3);
# df.plot(ax=ax, transform=ccrs.PlateCarree(), edgecolor='k', facecolor='none', linewidth=2);

In [ ]:
convert ../../../rgi7_data/l3_rgi7a_plots/global_map_surge.png -resize 20% ../../../rgi7_data/l3_rgi7a_plots/global_map_surge_small.jpeg

In [ ]:
# with sns.plotting_context('poster'), sns.axes_style('whitegrid'):

fac = 1.3
f = plt.figure(figsize=(12*fac, 7*fac))
ax = plt.axes(projection=ccrs.Robinson())
# ax.stock_img()
fname = '/home/users/fmaussion/www_fmaussion/misc/rgi7_data/l0_support_data/images/natural-earth-1_large2048px.png'
ax.imshow(imread(fname), origin='upper', transform=ccrs.PlateCarree(), 
          extent=[-180, 180, -90, 90])
handles = []

df.to_crs(ccrs.Robinson()).plot(ax=ax, edgecolor='k', facecolor='w', alpha=0.3);

for cat, col, name in zip([9, 1], ['royalblue', 'C3'], ['Not assigned', 'Marine-terminating']):
    s = rgi7.loc[rgi7.term_type == cat]
    if name == 'Not assigned':
        ax.scatter(s.cenlon.values, s.cenlat.values, fc=col, ec='None', s=1, transform=ccrs.PlateCarree(), linewidth=0);
    else:
        ax.scatter(s.cenlon.values, s.cenlat.values, fc=col, ec='k', s=15, transform=ccrs.PlateCarree(), linewidth=0.5);
        
    handles.append(mpatches.Patch(facecolor=col, label=name))


df.plot(ax=ax, transform=ccrs.PlateCarree(), edgecolor='k', facecolor='none', linewidth=1);

did10 = False
for i, r in df.iterrows():
    tx = r.min_x + 4
    ty = r.min_y 
    fs = 16
    t = r.o1region

    if t == '01':
        tx = r.min_x + 35
    if t == '03':
        tx = r.min_x + 15
        ty = r.min_y + 3
    if t == '05':
        tx = r.min_x + 40
    if t == '06':
        ty = r.min_y - 5.5
    if t == '08':
        tx = r.min_x + 20
    if t == '09':
        tx = r.min_x 
    if t == '11':
        tx = r.min_x + 24
    if t == '10':
        tx = r.min_x + 1
        ty = r.min_y + 15
        if not did10:
            did10 = True
            continue
    if t == '13':
        ty = r.min_y + 12
        tx = r.min_x + 30
    if t in ['14', '12', '16', '17']:
        tx = r.min_x 
    if t == '15':
        tx = r.min_x + 30
    if t == '17':
        ty = r.min_y + 26
    if t == '18':
        ty = r.min_y + 10
    if t == '19':
        ty = r.min_y + 10
    if t == '20':
        ty = r.min_y + 1
        tx = r.min_x - 20

    ax.text(tx, ty, t, transform=ccrs.PlateCarree(), 
            ha='left', va='bottom', fontsize=fs)


legend = ax.legend(handles=handles, loc='center left', fontsize=fs, title='Terminus type');
plt.setp(legend.get_title(),fontsize=fs+2)

plt.tight_layout();

plt.savefig('../../../rgi7_data/l3_rgi7a_plots/global_map_term_type.png', dpi=300, bbox_inches='tight')

# ax.scatter(dfr.lon.values, dfr.lat.values, edgecolor='k', facecolor='tab:cyan', s=24)

# df.plot(ax=ax, transform=ccrs.PlateCarree(), edgecolor='k', facecolor='w', alpha=0.3);
# df.plot(ax=ax, transform=ccrs.PlateCarree(), edgecolor='k', facecolor='none', linewidth=2);

In [ ]:
convert ../../../rgi7_data/l3_rgi7a_plots/global_map_term_type.png -resize 20% ../../../rgi7_data/l3_rgi7a_plots/global_map_term_type_small.jpeg

### Table statistics 

In [ ]:
adf = pd.DataFrame()
adf.loc['< 1 km²', 'RGI6 (N)'] = (rgi6['area'] < 1).sum()
adf.loc['< 1 km²', 'RGI6 (%)'] = (rgi6['area'] < 1).sum() / len(rgi6) * 100
adf.loc['< 1 km²', 'RGI7 (N)'] = (rgi7['area_km2'] < 1).sum()
adf.loc['< 1 km²', 'RGI7 (%)'] = (rgi7['area_km2'] < 1).sum() / len(rgi7) * 100
adf.loc['1-10 km²', 'RGI6 (N)'] = ((rgi6['area'] >= 1) & (rgi6['area'] < 10)).sum()
adf.loc['1-10 km²', 'RGI6 (%)'] = ((rgi6['area'] >= 1) & (rgi6['area'] < 10)).sum() / len(rgi6) * 100
adf.loc['1-10 km²', 'RGI7 (N)'] = ((rgi7['area_km2'] >= 1) & (rgi7['area_km2'] < 10)).sum()
adf.loc['1-10 km²', 'RGI7 (%)'] = ((rgi7['area_km2'] >= 1) & (rgi7['area_km2'] < 10)).sum() / len(rgi7) * 100
adf.loc['10-100 km²', 'RGI6 (N)'] = ((rgi6['area'] >= 10) & (rgi6['area'] < 100)).sum()
adf.loc['10-100 km²', 'RGI6 (%)'] = ((rgi6['area'] >= 10) & (rgi6['area'] < 100)).sum() / len(rgi6) * 100
adf.loc['10-100 km²', 'RGI7 (N)'] = ((rgi7['area_km2'] >= 10) & (rgi7['area_km2'] < 100)).sum()
adf.loc['10-100 km²', 'RGI7 (%)'] = ((rgi7['area_km2'] >= 10) & (rgi7['area_km2'] < 100)).sum() / len(rgi7) * 100
adf.loc['> 100 km²', 'RGI6 (N)'] = (rgi6['area'] >= 100).sum()
adf.loc['> 100 km²', 'RGI6 (%)'] = (rgi6['area'] >= 100).sum() / len(rgi6) * 100
adf.loc['> 100 km²', 'RGI7 (N)'] = (rgi7['area_km2'] >= 100).sum()
adf.loc['> 100 km²', 'RGI7 (%)'] = (rgi7['area_km2'] >= 100).sum() / len(rgi7) * 100
adf.loc['Total'] = adf.sum()
adf[['RGI6 (N)', 'RGI7 (N)']] = adf[['RGI6 (N)', 'RGI7 (N)']].astype(int)
adf[['RGI6 (%)', 'RGI7 (%)']] = adf[['RGI6 (%)', 'RGI7 (%)']].round(1)
adf.index.name = 'Area'
adf.columns = ['RGI 6.0 (N)', 'RGI 6.0 (%)', 'RGI 7.0 (N)', 'RGI 7.0 (%)']
adf

In [ ]:
print(adf.to_markdown())

In [ ]:
with sns.plotting_context('talk'), sns.axes_style('ticks'):

    bins = np.logspace(-2, 3, 100)

    h7, b7 = np.histogram(rgi7['area_km2'], bins=bins)
    h6, b6 = np.histogram(rgi6['area'], bins=bins)

    f, ax = plt.subplots(figsize=(10, 6))
    ax.plot(b6[:-1], h6, label='RGI 6.0');
    ax.plot(b7[:-1], h7, label='RGI 7.0');
    ax.set_xscale('log')
    ax.set_yscale('log')
    ax.set_ylim([7, 5e4])
    plt.legend()
    ax.set_title('Number of glaciers per size category (global)')
    ax.set_xlabel('Glacier area (km², logscale)');
    ax.set_ylabel('Glacier number (logscale)');

In [ ]:
with sns.plotting_context('talk'), sns.axes_style('ticks'):

    bins = np.logspace(-2, 3, 100)

    h7, b7 = np.histogram(rgi7['area_km2'], bins=bins)
    h6, b6 = np.histogram(rgi6['area'], bins=bins)

    f, ax = plt.subplots(figsize=(12, 7))
    ax.plot(b6[:-1], h6, label='RGI 6.0');
    ax.plot(b7[:-1], h7, label='RGI 7.0');
    ax.set_xscale('log')
    ax.set_yscale('log')
    plt.legend()
    ax.set_title('Number of glaciers per size category (global)')
    ax.set_xlabel('Glacier area (km², logscale)');
    ax.set_ylabel('Glacier number (logscale)');
    plt.savefig('../../../rgi7_data/l3_rgi7a_plots/global_histogram.png', dpi=300, bbox_inches='tight')

In [ ]:
with sns.plotting_context('talk'), sns.axes_style('ticks'):
    
    bins = np.logspace(-2, 2, 100)
    reg = ['13', '14', '15']
    reg = ['17']

    h7, b7 = np.histogram(rgi7.loc[rgi7.o1region.isin(reg)]['area_km2'], bins=bins)
    h6, b6 = np.histogram(rgi6.loc[rgi6.O1Region.isin(reg)]['area'], bins=bins)

    f, ax = plt.subplots(figsize=(10, 6))
    ax.plot(b6[:-1], h6, label='RGI 6.0');
    ax.plot(b7[:-1], h7, label='RGI 7.0');
    ax.set_xscale('log')
    ax.set_yscale('log')
    plt.legend()
    ax.set_title(f'Region 17 (Southern Andes)')
    ax.set_xlabel('Glacier area (km², logscale)');
    ax.set_ylabel('Glacier number (logscale)');

In [ ]:
rgi7['year'] = [int(y.split('-')[0]) for y in rgi7['src_date']]

In [ ]:
rgi6['year'] = [int(y[0:4]) for y in rgi6['BgnDate']]
rgi6['year'].unique()

In [ ]:
(rgi6['year'] == -999).sum()

In [ ]:
dy6 = np.abs(rgi6['year'] - 2000)
dy7 = np.abs(rgi7['year'] - 2000)

In [ ]:
ydf = pd.DataFrame()
ydf.loc['2000 ± 2 years', 'RGI6 (%)'] = (dy6 <= 2).sum() / len(rgi6) * 100
ydf.loc['2000 ± 2 years', 'RGI7 (%)'] = (dy7 <= 2).sum() / len(rgi7) * 100
ydf.loc['2000 ± 2-5 years', 'RGI6 (%)'] = ((dy6 <= 5) & (dy6 > 2)).sum() / len(rgi6) * 100
ydf.loc['2000 ± 2-5 years', 'RGI7 (%)'] = ((dy7 <= 5) & (dy7 > 2)).sum() / len(rgi7) * 100
ydf.loc['2000 ± 5-10 years', 'RGI6 (%)'] = ((dy6 <= 10) & (dy6 > 5)).sum() / len(rgi6) * 100
ydf.loc['2000 ± 5-10 years', 'RGI7 (%)'] = ((dy7 <= 10) & (dy7 > 5)).sum() / len(rgi7) * 100
ydf.loc['2000 ± > 10 years', 'RGI6 (%)'] = (dy6 > 10).sum() / len(rgi6) * 100
ydf.loc['2000 ± > 10 years', 'RGI7 (%)'] = (dy7 > 10).sum() / len(rgi7) * 100
ydf = ydf.round(1)
ydf.index.name = 'Outline year'
ydf.columns = ['RGI 6.0 (%)', 'RGI 7.0 (%)']
ydf

In [ ]:
print(ydf.to_markdown())

In [ ]:
len(rgi7) - len(rgi7.anlys_id.unique())

In [ ]:
fac = 1.3
f = plt.figure(figsize=(7*fac, 7*fac))
ax = plt.axes(projection=ccrs.Orthographic(-20, 45))

fname = '/home/users/fmaussion/www_fmaussion/misc/rgi7_data/l0_support_data/images/natural-earth-1_large2048px.png'
ax.imshow(imread(fname), origin='upper', transform=ccrs.PlateCarree(), 
          extent=[-180, 180, -90, 90]);

s = rgi7
handles = []
ax.scatter(s.cenlon.values, s.cenlat.values, color='C3', s=0.1, transform=ccrs.PlateCarree());
handles.append(mpatches.Patch(facecolor='C3', label=f"Glacier outlines"))
ax.legend(handles=handles, loc='center left', fontsize=fs);

plt.tight_layout();

In [ ]:
rgi7.utm_zone.unique()

In [ ]:
rgi7.columns

In [ ]:
rgi7.term_type.unique()

In [ ]:
rgi6.iloc[0]

## Attrs 

In [ ]:
rdf = pd.DataFrame(index=[0, 1, 2, 3, 9])
rdf.index.name = 'Value'
rdf['Terminus type'] = ['Land-terminating', 'Marine-terminating', 'Lake-terminating', 'Shelf-terminating', 'Not assigned']
rdf['RGI7 (N)'] = rgi7.groupby('term_type').count()['rgi_id']
rdf['RGI6 (N)'] = rgi6.groupby('TermType').count()['RGIId'].reset_index(drop=True)
rdf['RGI7 (Area)'] = rgi7.groupby('term_type')['area_km2'].sum().round(0).astype(int)
rdf['RGI6 (Area)'] = rgi6.groupby('TermType')['Area'].sum().round(0).reset_index(drop=True)
rdf = rdf.replace(np.NaN, 0)
rdf[['RGI7 (N)', 'RGI6 (N)']] = rdf[['RGI7 (N)', 'RGI6 (N)']].astype(int)
rdf[['RGI7 (Area)', 'RGI6 (Area)']] = rdf[['RGI7 (Area)', 'RGI6 (Area)']].astype(int)
rdf.columns = ['Terminus type', 'RGI 7.0 (N)', 'RGI 6.0 (N)', 'RGI 7.0 (Area)', 'RGI 6.0 (Area)']
rdf

In [ ]:
print(rdf.to_markdown())

In [ ]:
rgi6.groupby('Surging').count()['RGIId']

In [ ]:
rdf = pd.DataFrame(index=[0, 1, 2, 3, 9])
rdf.index.name = 'Value'
rdf['Surging'] = ['No evidence', 'Possible', 'Probable', 'Observed', 'Not assigned']
rdf['RGI7 (N)'] = rgi7.groupby('surge_type').count()['rgi_id']
rdf['RGI6 (N)'] = rgi6.groupby('Surging').count()['RGIId']
rdf['RGI7 (Area)'] = rgi7.groupby('surge_type')['area_km2'].sum().round(0).astype(int)
rdf['RGI6 (Area)'] = rgi6.groupby('Surging')['Area'].sum().round(0).astype(int)
rdf = rdf.replace(np.NaN, 0)
rdf[['RGI7 (N)', 'RGI6 (N)']] = rdf[['RGI7 (N)', 'RGI6 (N)']].astype(int)
rdf[['RGI7 (Area)', 'RGI6 (Area)']] = rdf[['RGI7 (Area)', 'RGI6 (Area)']].astype(int)
rdf.columns = ['Surge type', 'RGI 7.0 (N)', 'RGI 6.0 (N)', 'RGI 7.0 (Area)', 'RGI 6.0 (Area)']
rdf

In [ ]:
print(rdf.to_markdown())

In [ ]:
rgi7.groupby('surge_type').count()